### Find Original Distance (GAS-PIPELINES)

In [1]:
import pandas as pd
import numpy as np
import math
import json
import glob


import rasterio
import shapefile
from IPython.display import display

In [2]:
def deg2rad(deg):
    return (deg * (math.pi/180))

In [3]:
def get_distance(p, q):
    lat1, lon1 = p
    lat2, lon2 = q
    #const [lat1, lon1] = pair1, [lat2, lon2] = pair2;
    #const R = 6371; // Radius of the earth in km.

    R = 6371

    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a));

    return R * c

In [4]:
def find_distance(file_name):
    f = open(file_name)
    data = json.load(f)
    points = []
    for i in range(len(data['features'])):
        if data['features'][i]['geometry']['type'] == 'Point':
            points.append(data['features'][i]['geometry']['coordinates'])
    
    print(len(points))
    distance = 0
    for i in range(1, len(points)):
        p = points[i - 1]
        q = points[i]

        #distance += math.sqrt(math.pow((p[0] - q[0]), 2) + math.pow((p[1] - q[1]), 2))
        distance += get_distance(p, q)
    return distance

In [5]:
path = 'raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=300000.0, bottom=1890240.0, right=409800.0, top=2100000.0)
300000.0 1890240.0 409800.0 2100000.0


In [6]:
# reading LULC data
area_cover = 'mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

mask_data = area
print(mask_data.shape)
print(np.unique(mask_data, return_counts = True))

mask_data = mask_data.T

(20976, 10980)
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int16), array([     6905,  13870418,   7578815,  39715328, 104165063,  10435752,
        14235729,  40308470]))


In [11]:
index = 0
files = glob.glob(f'OUTPUTS/main_gas.geojson')
# print(files)
for f in files:
    dist = find_distance(f)
    name = files[index].split('.')[0].title()
    print(f'Distance = {dist} Kms, Pipeline = {name}')
    index = index + 1

84
Distance = 234.01070134678724 Kms, Pipeline = Outputs/Main_Gas


In [30]:
MAIN_KILO = dist
print(MAIN_KILO)

road_distance = dist

234.01070134678724


In [31]:
files = glob.glob('OUTPUTS/main_gas_prj.geojson')
print(files)

current_path_list = []
for file_name in files:

    f = open(file_name)
    data = json.load(f)
    # print(data)
    points = []
    for i in range(len(data['features'])):
        if data['features'][i]['geometry']['type'] == 'Point':
            points.append(data['features'][i]['geometry']['coordinates'])
    
    
    
    for i in range(1, len(points)):
        p = points[i - 1]

        current_path_list.append(p)

['OUTPUTS/main_gas_prj.geojson']


In [32]:
pixels_dict = {}
for i in range(len(current_path_list)):

    ext_i, ext_j = current_path_list[i]
    # print(ext_i, ext_j)

    pi, pj = (ext_i - left) / 10, ((ext_j - top) / 10) * -1
    pixels_dict[ext_i, ext_j] = [pi, pj]

In [33]:
classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

check_pixels = {
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
}


total_pxs = len(current_path_list)
for i in range(len(current_path_list)):
    ext_i, ext_j = current_path_list[i]
    pi, pj = pixels_dict[ext_i, ext_j]

    pi, pj = int(pi), int(pj)
    cs = mask_data[pi][pj]

    check_pixels[cs] += 1
    check_pixels[8] += 1


df_dict_pixels = {}
for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

for key, value in check_pixels.items():
    check_pixels[key] = np.round(((value / total_pxs) * 100), 2)

df_dict_perc = {}
for key, value in check_pixels.items():
    df_dict_perc[classes[key]] = f'{check_pixels[key]}%'

df_dict_dist = {}
for key, value in check_pixels.items():
    df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * road_distance) / 100)} Kms'


list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
display(route_stats)


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,1 pxs,1 pxs,11 pxs,49 pxs,4 pxs,1 pxs,16 pxs,83 pxs
ROAD PERCENTAGE,1.2%,1.2%,13.25%,59.04%,4.82%,1.2%,19.28%,100.0%
ROAD KILOMETERS,2.81 Kms,2.81 Kms,31.01 Kms,138.16 Kms,11.28 Kms,2.81 Kms,45.12 Kms,234.01 Kms


In [65]:
# path = 'raster.tif' 

# data = rasterio.open(path)
# print(data.bounds)

# extent = data.bounds

# left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
# print(left, bottom, right, top)

BoundingBox(left=300000.0, bottom=1890240.0, right=409800.0, top=2100000.0)
300000.0 1890240.0 409800.0 2100000.0


#### Route-Optimization Analysis

In [40]:
namming_dict = {
    1: 'A* Chebyshev distance',
    2: 'A* Diagonal distance',
    3: 'A* Euclidean distance',
    4: 'A* Manhattan distance',
    5: 'A* Proposed',
    6: 'Dijkstra\'s algorithm',
}

paths_dict = {
    1: 'paths/a-star-chesb',
    2: 'paths/a-star-diag',
    3: 'paths/a-star-euc',
    4: 'paths/a-star-man',
    5: 'paths/a-star-new',
    6: 'paths/dijk-route',
}

shape_dict = {
    1: 'routes-shape/a-star-chesb',
    2: 'routes-shape/a-star-diag',
    3: 'routes-shape/a-star-euc',
    4: 'routes-shape/a-star-man',
    5: 'routes-shape/a-star-new',
    6: 'routes-shape/dijk-route',
}

In [41]:
kilometers = []

In [42]:
for p in range(1, 7):
    path = paths_dict[p]
    path_list = []
    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()

    ordinates_dict = {}


    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        ordinates_dict[pi, pj] = [ext_i, ext_j]

    
    ordinates_dict
    ordinates_list = []
    for key, value in ordinates_dict.items():
        ordinates_list.append(value)
    
    road_lenght = 0
    index = 1
    for _ in ordinates_list[1:]:
        i, j = ordinates_list[index][0], ordinates_list[index][1]
        x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

        if (x - i == 10.0) and (y - j == -10.0):
            road_lenght += math.sqrt(2 * 100)
        else:
            road_lenght += 10 
    
    print(f'Road-length ALGO = [{namming_dict[p]}] = {road_lenght / 1000} kms')
    kilometers.append(road_lenght / 1000)

Road-length ALGO = [A* Chebyshev distance] = 182.37 kms
Road-length ALGO = [A* Diagonal distance] = 191.44 kms
Road-length ALGO = [A* Euclidean distance] = 196.09 kms
Road-length ALGO = [A* Manhattan distance] = 188.04 kms
Road-length ALGO = [A* Proposed] = 193.5 kms
Road-length ALGO = [Dijkstra's algorithm] = 207.25 kms


In [44]:
# reading LULC data
area_cover = 'mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

mask_data = area
print(mask_data.shape)
print(np.unique(mask_data, return_counts = True))

mask_data = mask_data.T

(20976, 10980)
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int16), array([     6905,  13870418,   7578815,  39715328, 104165063,  10435752,
        14235729,  40308470]))


In [48]:
for k in range(1, 7):

    path = paths_dict[k]
    path_list = []
    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()
    classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

    pixels_dict = {}
    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        # ordinates_dict[pi, pj] = [ext_i, ext_j]
        pixels_dict[ext_i, ext_j] = [pi, pj]

    check_pixels = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
    }

    df_dict_pixels = {}

    r = shapefile.Reader(f'OUTPUTS/{shape_dict[k]}/shapefiles/test/multipoint.shp')
    feature = r.shapeRecords()[0]
    first = feature.shape.__geo_interface__  

    total_pxs = len(first['coordinates'])
    for cord in first['coordinates']:
        pi, pj = pixels_dict[cord]
        cs = mask_data[pi][pj]

        check_pixels[cs] += 1
        check_pixels[8] += 1
    
    for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    print(str)

    print(f'ALGORITHM = [{namming_dict[k]}]')
    # print(f'PIXELS LULC {check_pixels}')
    # print(f'PIXELS LULC {df_dict_pixels}')

    for key, value in check_pixels.items():
        check_pixels[key] = np.round(((value / total_pxs) * 100), 2)
    
    # print(f'PERCENTAGE LULC {check_pixels}')

    classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}
    df_dict_perc = {}
    for key, value in check_pixels.items():
        df_dict_perc[classes[key]] = f'{check_pixels[key]}%'
    
    # print(f'ROAD PERCENTAGE = {df_dict_perc}')

    road_distance = MAIN_KILO
    current_road_distance = kilometers[k - 1]
    # print(f'CURRENT DISTANCE = {road_distance}, OPTIMIZED DISTANCE = {kilometers[k - 1]}')

    df_dict_dist = {}
    recheck = 0
    for key, value in check_pixels.items():
        df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * current_road_distance) / 100)} Kms'
        recheck += (check_pixels[key] * road_distance) / 100
    
    # print(f'ROAD KILOMETERS = {df_dict_dist}')
    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
    route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
    display(route_stats)

    print(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
    
    print(str)
            

-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Chebyshev distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,367 pxs,143 pxs,4622 pxs,9025 pxs,1278 pxs,417 pxs,2386 pxs,18238 pxs
ROAD PERCENTAGE,2.01%,0.78%,25.34%,49.48%,7.01%,2.29%,13.08%,100.0%
ROAD KILOMETERS,3.67 Kms,1.42 Kms,46.21 Kms,90.24 Kms,12.78 Kms,4.18 Kms,23.85 Kms,182.37 Kms


CURRENT DISTANCE = 234.0 Kms, OPTIMIZED DISTANCE = 182.37 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Diagonal distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,496 pxs,133 pxs,4962 pxs,9215 pxs,1408 pxs,418 pxs,2513 pxs,19145 pxs
ROAD PERCENTAGE,2.59%,0.69%,25.92%,48.13%,7.35%,2.18%,13.13%,100.0%
ROAD KILOMETERS,4.96 Kms,1.32 Kms,49.62 Kms,92.14 Kms,14.07 Kms,4.17 Kms,25.14 Kms,191.44 Kms


CURRENT DISTANCE = 234.0 Kms, OPTIMIZED DISTANCE = 191.44 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Euclidean distance]


...

CURRENT DISTANCE = 234.0 Kms, OPTIMIZED DISTANCE = 188.04 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Proposed]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,475 pxs,157 pxs,4523 pxs,9122 pxs,2209 pxs,457 pxs,2408 pxs,19351 pxs
ROAD PERCENTAGE,2.45%,0.81%,23.37%,47.14%,11.42%,2.36%,12.44%,100.0%
ROAD KILOMETERS,4.74 Kms,1.57 Kms,45.22 Kms,91.22 Kms,22.1 Kms,4.57 Kms,24.07 Kms,193.5 Kms


CURRENT DISTANCE = 234.0 Kms, OPTIMIZED DISTANCE = 193.5 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [Dijkstra's algorithm]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,145 pxs,4 pxs,2611 pxs,16691 pxs,16 pxs,0 pxs,1259 pxs,20726 pxs
ROAD PERCENTAGE,0.7%,0.02%,12.6%,80.53%,0.08%,0.0%,6.07%,100.0%
ROAD KILOMETERS,1.45 Kms,0.04 Kms,26.11 Kms,166.9 Kms,0.17 Kms,0.0 Kms,12.58 Kms,207.25 Kms


CURRENT DISTANCE = 234.0 Kms, OPTIMIZED DISTANCE = 207.25 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-


In [98]:
# list_of_dicts = [
#     df_dict_pixels,
#     df_dict_perc,
#     df_dict_dist,
# ]

In [104]:
# route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])

In [107]:
# route_stats

,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland
PIXELS LULC,367 pxs,143 pxs,4622 pxs,9025 pxs,1278 pxs,417 pxs,2386 pxs
ROAD PERCENTAGE,2.01%,0.78%,25.34%,49.48%,7.01%,2.29%,13.08%
ROAD KILOMETERS,4.7 Kms,1.83 Kms,59.3 Kms,115.79 Kms,16.4 Kms,5.36 Kms,30.61 Kms
